### **SCRIPT QUE GERA UM CSV A APARTIR DOS DADOS ACUMULADOS POR CIDADE NO PIAUÍ**

Felipe Dantas

In [1]:
import gspread
import pandas as pd
import numpy as np
import datetime

#### **DADOS COLETADOS DIRETAMENTO DA PLANILHA NO GOOGLE SHEET**
##### Autenticação será exigida apenas na primeira vez

In [2]:
gc = gspread.oauth()

#### Carregando a planilha completa do Governo do Piauí pela seguinte url:
<https://docs.google.com/spreadsheets/d/1b-GkDhhxJIwWcA6tk3z4eX58f-f1w2TA2f2XrI4XB1w/edit#gid=1514947706>

In [4]:
url = 'https://docs.google.com/spreadsheets/d/1b-GkDhhxJIwWcA6tk3z4eX58f-f1w2TA2f2XrI4XB1w/edit#gid=1514947706'
sh = gc.open_by_url(url)

#### Lista todas as planilhas

In [62]:
worksheet_list = sh.worksheets()

In [63]:
for e in enumerate(worksheet_list):
    print(e)

(0, <Worksheet 'Dt Atualiza' id:1514947706>)
(1, <Worksheet 'Confirmados' id:1869243391>)
(2, <Worksheet 'Descartados' id:1183637221>)
(3, <Worksheet 'Altas Médicas' id:921393660>)
(4, <Worksheet 'Óbitos' id:1820834527>)
(5, <Worksheet 'TR' id:1934173117>)
(6, <Worksheet 'Invest Laborat' id:75671645>)
(7, <Worksheet 'Invest Lab Dia' id:889562226>)
(8, <Worksheet 'Conf Obito Mun' id:532454257>)
(9, <Worksheet 'Tab Apoio' id:1340447443>)
(10, <Worksheet '%Mun Casos' id:671234104>)
(11, <Worksheet '% Mun Obt' id:1008596386>)
(12, <Worksheet 'Conf Sexo' id:0>)
(13, <Worksheet 'Casos por Dia' id:181109212>)
(14, <Worksheet 'Óbito Sexo' id:953543508>)
(15, <Worksheet 'Conf Obt Fx Etária' id:1402178422>)
(16, <Worksheet 'Obt MM60' id:927613120>)
(17, <Worksheet 'Comorbidades' id:687147050>)
(18, <Worksheet 'Mapa' id:1503162102>)
(19, <Worksheet 'Internação' id:1700633974>)
(20, <Worksheet 'Tx Ocup LC' id:193507430>)
(21, <Worksheet 'Tx Ocup UTI' id:277286516>)
(22, <Worksheet 'Tx Ocup LE' id:

#### SELECIONANDO WORKSHEET PELO INDICE OU PELO NOME

In [64]:
#sh.get_worksheet(8)

##### PLANILHA COM DATA DA ÚLTIMA ATUALIZAÇÃO DOS DADOS

In [65]:
worksheet_last_update = sh.worksheet('Dt Atualiza')

##### PLANILHA DE CASOS ACUMULADOS POR MUNICÍPIOS

In [66]:
worksheet_acumulado_muni = sh.worksheet('Conf Obito Mun')

##### OBTENDO TODOS OS VALORES DAS WORKSHEET SELECIONADOS E TRANSFORMANDO EM ARRAY.

DATA E HORA DA ÚLTIMA ATUALIZAÇÃO DOS DADOS

Necessário para registrar o dia das ocorrencias.

In [67]:
last_update_arr = np.array(worksheet_last_update.get_all_values())

In [68]:
last_update = last_update_arr[1][0].split(' ')
del last_update[:3]
del last_update[1]
last_update[-1] = last_update[-1].replace('h',':')

In [69]:
last_update

['12/05/2020', '19:00']

DADOS ACUMULADOS DE CASOS POR MUNICÍPIOS

Daqui iremos extrair informações de comfirmados e óbitos por município

In [70]:
acumulado_muni = np.array(worksheet_acumulado_muni.get_all_values())

In [71]:
acumulado_muni[:10]

array([['id', 'Município', 'Confirmados', 'Óbitos', 'Incidência',
        'População', 'CEP'],
       ['1', 'AGUA BRANCA', '26', '2', '14,93', '17411', '64460000'],
       ['2', 'ALTO LONGA', '1', '0', '0,70', '14304', '64360000'],
       ['3', 'ALTOS', '15', '0', '3,70', '40524', '64290000'],
       ['4', 'ALVORADA DO GURGUEIA', '1', '0', '1,85', '5419',
        '64923000'],
       ['5', 'AMARANTE', '3', '0', '1,70', '17598', '64400000'],
       ['6', 'ANGICAL DO PIAUI', '3', '0', '4,42', '6788', '64410000'],
       ['7', 'ANISIO DE ABREU', '9', '0', '9,11', '9880', '64780000'],
       ['8', 'AROAZES', '1', '0', '1,71', '5835', '64310000'],
       ['9', 'BAIXA GRANDE DO RIBEIRO', '5', '1', '4,32', '11586',
        '64868000']], dtype='<U26')

#### TRANSFORMANDO OS ARRAYS PARA DATAFRAME PANDAS

DATAFRAME DA ULTIMA ATUALIZAÇÃO

In [72]:
df_last_update = pd.DataFrame(last_update).T

In [73]:
df_last_update.columns = ['data', 'hora']

In [74]:
df_last_update['data_ult_atualizacao'] = pd.to_datetime(df_last_update['data'] + ' ' + df_last_update['hora'], format="%d/%m/%Y %H:%M")

In [75]:
df_last_update

,data,hora,data_ult_atualizacao
0,12/05/2020,19:00,2020-05-12 19:00:00


DATAFRAME DE DADOS ACUMULADOS POR MUNICÍPIOS

In [76]:
df_acumulado_muni = pd.DataFrame(acumulado_muni[1:], columns=acumulado_muni[0])

In [77]:
df_acumulado_muni.tail()

,id,Município,Confirmados,Óbitos,Incidência,População,CEP
88,89,VALENCA DO PIAUI,1,1,"0,48",20918,64300000
89,90,VARZEA GRANDE,1,0,"2,28",4391,64525000
90,91,VERA MENDES,2,0,"6,50",3077,64568000
91,92,VILA NOVA DO PIAUI,2,0,"6,73",2971,64688000
92,,PIAUÍ,1612,57,"49,25",3273227,


Excluindo a última linha, pois esta última registra o somatório total de casos

In [78]:
df_acumulado_muni.drop(index=df_acumulado_muni.last_valid_index(), inplace=True)

CONVERTENDO OS TIPOS DE DADOS DO DATAFRAME DE DADOS ACUMULADOS POR MUNICÍPIOS

In [79]:
df_acumulado_muni['Município'] = df_acumulado_muni['Município'].astype('string')

In [80]:
df_acumulado_muni['Confirmados'] = df_acumulado_muni['Confirmados'].astype('int')

In [81]:
df_acumulado_muni['Óbitos'] = df_acumulado_muni['Óbitos'].astype('int')

###### Na coluna 'Incidência' a virgula é utilizada com separador de valores decimais. Necessário substituir antes

In [82]:
df_acumulado_muni['Incidência'] = df_acumulado_muni['Incidência'].apply(lambda val: float(val.replace(',','.')))

In [83]:
df_acumulado_muni['População'] = df_acumulado_muni['População'].astype('int')

In [84]:
df_acumulado_muni['CEP'] = df_acumulado_muni['CEP'].astype('int')

### **IMPORTANDO CSV COM A BASE OFICIAL DO IBGE PARA OS MUNICÍPIOS DO PIAUÍ**

In [106]:
df_municipios = pd.read_csv("input/municipios_piaui.csv", encoding='utf-8')

In [107]:
df_municipios = df_municipios.drop(columns={'NM_MESO',
                                            'CD_GEOCME',
                                            'NM_REGIAO'}
                                  ).sort_values('NM_MUNICIP').reset_index(drop=True)

In [108]:
df_municipios.columns = df_municipios.columns.str.lower()

In [109]:
df_municipios.rename(columns={'nm_municip':'nm_cidade',
                              'cd_geocmu':'id_cidade',
                              'cd_geocmi':'id_micro',
                              'cd_geocuf':'id_estado',
                              'populacao_estimada_2019':'pop2019_est'}, inplace=True)

In [110]:
df_municipios.tail()

,nm_cidade,id_cidade,nm_micro,id_micro,nm_estado,id_estado,lng,lat,pop2019_est
219,VILA NOVA DO PIAUÍ,2211605,ALTO MÉDIO CANINDÉ,22015,PIAUÍ,22,-40.936151,-7.202121,2971
220,VÁRZEA BRANCA,2211357,SÃO RAIMUNDO NONATO,22011,PIAUÍ,22,-42.921593,-9.322704,4947
221,VÁRZEA GRANDE,2211407,VALENÇA DO PIAUÍ,22006,PIAUÍ,22,-42.225271,-6.553657,4391
222,WALL FERRAZ,2211704,PICOS,22013,PIAUÍ,22,-41.877806,-7.265636,4462
223,ÁGUA BRANCA,2200202,MÉDIO PARNAÍBA PIAUIENSE,22005,PIAUÍ,22,-42.628069,-5.913995,17411


## CÓDIGO PARA FAZER A CORREÇÃO DOS NOMES ERRADOS DA BASE DO GOVERNO DO PIAUÍ

LUIS HENRIQUE

## Funções para limpeza dos dados do governo

In [111]:
from unicodedata import normalize

In [112]:
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [113]:
def numero_letras_erradas_menor_dois(cidade1, cidade2):
    diferenca = 0
    if len(cidade1) != len(cidade2):
        return False
    for i, j in list(zip(list(cidade1), list(cidade2))):
        if i != j:
            diferenca += 1
    return diferenca < 2

In [114]:
def compara_cidades(cidade1, cidade2):
    if cidade1.lower() == cidade2.lower():
        return True
    elif remover_acentos(cidade1.lower()) == remover_acentos(cidade2.lower()):
        return True
    elif numero_letras_erradas_menor_dois(remover_acentos(cidade1.lower()), remover_acentos(cidade2.lower())):
        return True
    else: 
        return False

Gerar o numero de casos e óbitos para cada cidade

In [115]:
confirmados = [0 for i in range(len(df_municipios['nm_cidade']))] # Gera um lista com 224 itens
obitos = [0 for i in range(len(df_municipios['nm_cidade']))] # Gera um lista com 224 itens

for itrIB, cIBGE in enumerate(df_municipios['nm_cidade']): 
    for itrCO, cCOVID in enumerate(df_acumulado_muni['Município']):
        if compara_cidades(cIBGE, cCOVID):
            confirmados[itrIB] = df_acumulado_muni['Confirmados'].iloc[itrCO]
            obitos[itrIB] = df_acumulado_muni['Óbitos'].iloc[itrCO]

#### ADICIONANDO INFORMAÇÕES DE CASOS ACUMULADOS (CONFIRMADOS E ÓBITOS) E DATA AO DATAFRAME DE MUNICÍPIOS DO IBGE

In [116]:
df_municipios['confirmados_acum'] = confirmados
df_municipios['obitos_acum'] = obitos

In [117]:
for dt in df_municipios['nm_cidade']:
    df_municipios['data'] = df_last_update['data_ult_atualizacao'][0]

#### REORDENANDOS AS COLUNAS

In [118]:
df_municipios = df_municipios[['data','id_cidade','nm_cidade','confirmados_acum','obitos_acum','pop2019_est','id_micro','nm_micro','id_estado','nm_estado','lng','lat']]

VERIFICANDO O TIPO DOS DADOS

In [119]:
df_municipios.dtypes

data                datetime64[ns]
id_cidade                    int64
nm_cidade                   object
confirmados_acum             int64
obitos_acum                  int64
pop2019_est                  int64
id_micro                     int64
nm_micro                    object
id_estado                    int64
nm_estado                   object
lng                        float64
lat                        float64
dtype: object

BUSCANDO TODOS OS MUNICÍPIOS COM CASOS CONFIRMADOS MAIORES QUE ZERO

In [120]:
df_municipios.query('confirmados_acum > 0').sort_values(['confirmados_acum'], ascending=False).reset_index(drop=True)

,data,id_cidade,nm_cidade,confirmados_acum,obitos_acum,pop2019_est,id_micro,nm_micro,id_estado,nm_estado,lng,lat
0,2020-05-12 19:00:00,2211001,TERESINA,937,26,864845,22003,TERESINA,22,PIAUÍ,-42.740648,-5.102260
1,2020-05-12 19:00:00,2207702,PARNAÍBA,89,4,153078,22002,LITORAL PIAUIENSE,22,PIAUÍ,-41.753544,-2.959207
2,2020-05-12 19:00:00,2208007,PICOS,75,3,78222,22013,PICOS,22,PIAUÍ,-41.522312,-7.058929
3,2020-05-12 19:00:00,2203701,ESPERANTINA,39,0,39737,22001,BAIXO PARNAÍBA PIAUIENSE,22,PIAUÍ,-42.189690,-3.823370
4,2020-05-12 19:00:00,2202208,CAMPO MAIOR,35,0,46833,22004,CAMPO MAIOR,22,PIAUÍ,-42.144995,-4.876786
...,...,...,...,...,...,...,...,...,...,...,...,...
87,2020-05-12 19:00:00,2208650,QUEIMADA NOVA,1,0,8992,22015,ALTO MÉDIO CANINDÉ,22,PIAUÍ,-41.351176,-8.516010
88,2020-05-12 19:00:00,2208700,REDENÇÃO DO GURGUÉIA,1,0,8778,22010,ALTO MÉDIO GURGUÉIA,22,PIAUÍ,-44.454712,-9.524275
89,2020-05-12 19:00:00,2208809,REGENERAÇÃO,1,0,17978,22005,MÉDIO PARNAÍBA PIAUIENSE,22,PIAUÍ,-42.490414,-6.352495
90,2020-05-12 19:00:00,2208874,RIBEIRA DO PIAUÍ,1,0,4477,22015,ALTO MÉDIO CANINDÉ,22,PIAUÍ,-42.613564,-7.927756


In [121]:
df_municipios['tx_incidencia'] = round((df_municipios['confirmados_acum'] / df_municipios['pop2019_est']) * 100000, 2)

In [122]:
df_municipios['tx_mortalidade'] = round((df_municipios['obitos_acum'] / df_municipios['pop2019_est'] * 100000), 2)

In [123]:
df_municipios['coef_letalidade'] = df_municipios['obitos_acum'] / df_municipios['confirmados_acum']

In [124]:
df_municipios.query('obitos_acum > 0').head()

,data,id_cidade,nm_cidade,confirmados_acum,obitos_acum,pop2019_est,id_micro,nm_micro,id_estado,nm_estado,lng,lat,tx_incidencia,tx_mortalidade,coef_letalidade
16,2020-05-12 19:00:00,2201150,BAIXA GRANDE DO RIBEIRO,5,1,11586,22007,ALTO PARNAÍBA PIAUIENSE,22,PIAUÍ,-45.160980,-8.505753,43.16,8.63,0.2
30,2020-05-12 19:00:00,2201919,BOM PRINCÍPIO DO PIAUÍ,1,1,5630,22002,LITORAL PIAUIENSE,22,PIAUÍ,-41.630631,-3.157407,17.76,17.76,1.0
35,2020-05-12 19:00:00,2202000,BURITI DOS LOPES,2,1,19781,22002,LITORAL PIAUIENSE,22,PIAUÍ,-41.803968,-3.239414,10.11,5.06,0.5
47,2020-05-12 19:00:00,2202307,CANTO DO BURITI,2,1,21187,22011,SÃO RAIMUNDO NONATO,22,PIAUÍ,-43.337609,-8.342575,9.44,4.72,0.5
55,2020-05-12 19:00:00,2202703,COCAL,2,1,27787,22002,LITORAL PIAUIENSE,22,PIAUÍ,-41.542303,-3.453676,7.20,3.60,0.5


## Exportando CSV

In [199]:
df_muni_subset.loc[0, 'data']

Timestamp('2020-05-11 18:40:00')

In [125]:
df_municipios.to_csv('output/dados_piaui_covid.csv', index=False, encoding='utf-8')